In [1]:
from untils import load_data
import numpy as np
import models 

In [2]:
data_file = 'D:/zhuomian/file/弱网中鲁棒性的恶意节点检测/exp/dataset/20241106_testTX_constantUniform_simtime-100_num_nodes-50_BHradio-0_SFradio-0_fieldLength-20_uniform-1/ProcessedMonitorSnifferRx.csv'

In [3]:
np.set_printoptions(threshold=np.inf)  # 这将打印所有内容，去掉折叠
processed_data, e_adj, T, group_num = load_data(1,data_file)

Total number of edges (undirected graph): 484
Total number of edges (undirected graph): 439
Total number of edges (undirected graph): 77


In [4]:
# 设置numpy的打印选项，显示更多行列


# 打印节点特征（processed_data.x）
# print(T)

In [5]:
# 打印边连接关系（processed_data.edge_index）
# print(processed_data[0].edge_index.numpy())

In [6]:
# 打印节点标签（processed_data.y）
# print(processed_data[0].edge_weights.numpy())
# print(processed_data[0].x[:,0:8].numpy())

In [7]:
train_model = models.HypergraphModel(8, 3)
print(train_model)

HypergraphModel(
  (layer1): HypergraphConvLayer(
    (conv): HypergraphConv(8, 64)
  )
  (layer2): HypergraphConvLayer(
    (conv): HypergraphConv(64, 32)
  )
  (edge_conv): GraphConvolution()
  (class_classifier): Sequential(
    (0): Linear(in_features=32, out_features=32, bias=True)
    (1): ReLU()
    (2): Linear(in_features=32, out_features=2, bias=True)
    (3): Softmax(dim=1)
  )
)


In [8]:
import time
import torch.optim as optim
t = time.time()
train_model.train()
learning_rate = 0.001
weight_decay = 5e-4
optimizer = optim.Adam(train_model.parameters(), lr=learning_rate, weight_decay=weight_decay)
optimizer.zero_grad()
# print(processed_data[0].x[0:8].shape)
# print(processed_data[0].edge_weights.shape)
# print(processed_data[0].edge_index.shape)
output = train_model(processed_data[0].x[:, 0:8], processed_data[0].edge_index, processed_data[0].edge_weights, processed_data[0].x[:, 8:11], e_adj[0], T[0])

loss_train = criteria(output[idx_train], labels[idx_train])
acc_train = acc_measure(output[idx_train], labels[idx_train])

loss_train.backward()
optimizer.step()



x.shape
torch.Size([484, 32])
node_features.shape
torch.Size([484, 32])
edge_features.shape
torch.Size([484, 3])
adj_e.shape
torch.Size([233, 233])
T.shape
torch.Size([50, 233])


RuntimeError: addmm: Argument #3 (dense): Expected dim 0 size 50, got 484